In [1]:
import gymnasium as gym
import numpy as np
import random

### Exercice 1 : Initialisation de l’environnement et des structures de données

In [2]:
env = gym.make("Taxi-v3")

num_state = env.observation_space.n
num_action = env.action_space.n
policy_table = np.ones((num_state, num_action)) / num_action
value_table = np.zeros(num_state)


print("les lignes de la table de politique :\n", policy_table[:2])
print("les valeurs de la table de valeurs :\n", value_table[:2])

les lignes de la table de politique :
 [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]
les valeurs de la table de valeurs :
 [0. 0.]


### Exercice 2 : Exploration et collecte d’épisodes

In [3]:
num_episodes = 20

for episode in range(num_episodes):
    state = env.reset()[0]  
    done = False
    total_reward = 0

    print(f"\nÉpisode {episode + 1}")

    while not done:
        action = env.action_space.sample()  
        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        print(f"Action : {action}, Récompense : {reward}")

    print(f"Récompense totale : {total_reward}")

env.close()


Épisode 1
Action : 3, Récompense : -1
Action : 0, Récompense : -1
Action : 2, Récompense : -1
Action : 0, Récompense : -1
Action : 5, Récompense : -10
Action : 5, Récompense : -10
Action : 5, Récompense : -10
Action : 1, Récompense : -1
Action : 4, Récompense : -10
Action : 4, Récompense : -10
Action : 0, Récompense : -1
Action : 2, Récompense : -1
Action : 0, Récompense : -1
Action : 5, Récompense : -10
Action : 5, Récompense : -10
Action : 1, Récompense : -1
Action : 2, Récompense : -1
Action : 0, Récompense : -1
Action : 4, Récompense : -10
Action : 0, Récompense : -1
Action : 5, Récompense : -10
Action : 0, Récompense : -1
Action : 0, Récompense : -1
Action : 5, Récompense : -10
Action : 4, Récompense : -10
Action : 4, Récompense : -10
Action : 2, Récompense : -1
Action : 5, Récompense : -10
Action : 1, Récompense : -1
Action : 2, Récompense : -1
Action : 4, Récompense : -10
Action : 3, Récompense : -1
Action : 3, Récompense : -1
Action : 3, Récompense : -1
Action : 5, Récompense 

### Exercice 3 : Mise à jour de la politique avec PPO

In [4]:
gamma = 0.99          
lr_policy = 0.1      
clip_epsilon = 0.2    

def update_ppo(episode_states, episode_actions, episode_rewards):

    discounted_rewards = []
    R = 0
    for r in reversed(episode_rewards):
        R = r + gamma * R
        discounted_rewards.insert(0, R)

    
    discounted_rewards = (discounted_rewards - np.mean(discounted_rewards)) / (np.std(discounted_rewards) + 1e-8)

    
    for state, action, G in zip(episode_states, episode_actions, discounted_rewards):
        
        advantage = G - value_table[state]

       
        old_prob = policy_table[state][action]
        new_prob = old_prob * np.exp(lr_policy * advantage)

       
        ratio = new_prob / old_prob
        clipped_ratio = np.clip(ratio, 1 - clip_epsilon, 1 + clip_epsilon)
        policy_table[state][action] = old_prob + lr_policy * (clipped_ratio - 1) * advantage

        
        policy_table[state] /= np.sum(policy_table[state])

    
        value_table[state] += lr_policy * advantage


episode_states = [0, 1, 2]  
episode_actions = [1, 2, 0] 
episode_rewards = [1, -1, 2] 
update_ppo(episode_states, episode_actions, episode_rewards)

### Exercice 4 : Évaluation de l'agent après entraînement

In [ ]:
def evaluate_agent(num_episodes=20):
    total_rewards = []
    for _ in range(num_episodes):
        state, _ = env.reset() 
        done = False
        episode_reward = 0

        while not done:
            action = np.argmax(policy_table[state])  
            state, reward, done, _, _ = env.step(action)
            episode_reward += reward

        total_rewards.append(episode_reward)

    print(f"Récompenses moyennes sur {num_episodes} épisodes: {np.mean(total_rewards)}")
    return total_rewards


for episode in range(100):  
    states, actions, rewards = [], [], []
    state, _ = env.reset()  
    done = False

    while not done:
        action = np.random.choice(num_action, p=policy_table[state])
        next_state, reward, done, _, _ = env.step(action)

        states.append(state)
        actions.append(action)
        rewards.append(reward)
        state = next_state

    update_ppo(states, actions, rewards)

print("\n=== Performances après entraînement ===")
evaluate_agent()


=== Performances après entraînement ===
